In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mygene
import collections
import math
from sklearn.decomposition import PCA
from scipy import stats

In [2]:
# import pip
# pip.main(['install','mygene'])

# 1. GDSC data processing

## 1.1 GDSC drug sensitivity data binarized

### 1.1.1 read GDSC drug sensitivity data 

In [3]:
gdscDrugFile = "GDSC_original_data/GDSC1_fitted_dose_response_25Feb20.xlsx"
dropIndexList = [687794, 688015, 753597, 905943, 906825, 907063, 908118, 908120, 908134, 908442, 908480, \
             909762, 910918, 924189, 930301, 946363, 949092, 949154, 1240149, 1240157, 1240158, 1240220, \
             1240224, 1298144, 1298167, 1298218, 1299069, 1303901, 1303911, 1330960, 1330981, 1331030, \
             1480367, 1789883] # These cell lines are droped for GDSC1 drug response data due to data incompleteness
responseOutput = "processed_data/GDSC1_drug_response_binaryAUC.csv"


In [4]:
gdscDrugData = pd.read_excel(gdscDrugFile, header = 0)
print("The number of cell lines with drug sensitivity data: " + str(len(gdscDrugData.CELL_LINE_NAME.unique())))
print("The number of drugs: " + str(len(gdscDrugData.DRUG_ID.unique())))
gdscDrugData.head()

The number of cell lines with drug sensitivity data: 987
The number of drugs: 367


,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,PATHWAY_NAME,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE
0,GDSC1,281,12974350,683665,MC-CAR,SIDM00636,MM,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,2.395685,0.982114,0.022521,-0.189576
1,GDSC1,281,12975300,684055,ES3,SIDM00265,UNCLASSIFIED,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,3.140923,0.984816,0.031840,0.508635
2,GDSC1,281,12975647,684057,ES5,SIDM00263,UNCLASSIFIED,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,3.968757,0.985693,0.026052,1.284229
3,GDSC1,281,12975980,684059,ES7,SIDM00269,UNCLASSIFIED,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,2.692768,0.972699,0.110056,0.088760
4,GDSC1,281,12976330,684062,EW-11,SIDM00203,UNCLASSIFIED,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,2.478678,0.944462,0.087011,-0.111820


### 1.1.2 obtain the AUC value

In [5]:
index = sorted(gdscDrugData.COSMIC_ID.unique())
columns = sorted(gdscDrugData.DRUG_ID.unique())
tmpInd = 0
cellidDic = {}
for ind in index:
    cellidDic[ind] = tmpInd
    tmpInd += 1
tmpInd = 0
drugnameDic = {}
for col in columns:
    drugnameDic[col] = tmpInd
    tmpInd += 1
drugResponse = np.empty((len(index), len(columns)))
drugResponse[:,:] = np.nan
for ind, row in gdscDrugData.iterrows():
    cellidInd = cellidDic[row["COSMIC_ID"]]
    drugnameInd = drugnameDic[row["DRUG_ID"]]
    auc = row["AUC"]
    drugResponse[cellidInd][drugnameInd] = auc
drugResponseDF = pd.DataFrame(drugResponse, index = index, columns = columns)
print(drugResponseDF.shape)
drugResponseDF.head()

(987, 367)


,1,3,5,6,9,11,17,29,30,32,...,1494,1495,1496,1497,1498,1502,1526,1527,1529,1530
683665,0.982114,0.980891,0.903979,0.986077,0.937027,0.728050,0.978113,0.945160,0.988067,0.897546,...,0.650553,0.984669,0.935993,0.987922,0.979853,0.982706,0.928560,0.859750,NaN,NaN
683667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.685058,NaN,0.941059,NaN,NaN,NaN,NaN,NaN,NaN,NaN
684052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.290396,0.784440,0.664747,0.983333,0.919021,0.983979,0.921035,0.655726,0.621239,0.984947
684055,0.984816,0.981415,0.941416,0.981549,0.956082,0.975094,0.979193,0.926046,0.924635,0.976683,...,0.296371,0.803190,0.671472,0.961318,0.973103,0.974498,0.963667,0.816016,0.818002,0.983505
684057,0.985693,0.980594,0.951725,0.961611,0.917021,0.981629,0.981101,0.934873,0.963014,0.982634,...,0.391312,0.854097,0.854424,0.950922,0.961995,0.967201,0.976020,0.596586,0.796861,0.973428


In [6]:
drugResponseDF = drugResponseDF.drop(dropIndexList)
print(drugResponseDF.shape)

(953, 367)


### 1.1.3 Binarize response value with the waterfall method

In [7]:
def lineFromPoints(P,Q): 
    a = Q[1] - P[1] 
    b = P[0] - Q[0]  
    c = a*(P[0]) + b*(P[1])  
    return a, b, c

In [8]:
drugThreshold = {}
for drug in drugResponseDF.columns:
    resVector = drugResponseDF[drug].values
    resVector = resVector[~np.isnan(resVector)] 
    n = resVector.shape[0]
    y = np.sort(resVector)
    x = np.array([i for i in range(1, n + 1)])
    pr = stats.pearsonr(y, x)[0]
    if pr > 0.95: # Use the median as the threshold if the response curve is close to a straight line
        drugThreshold[drug] = np.median(y)
    else: # Use the waterfall method for finding the threshold
        a, b, c = lineFromPoints([x[0], y[0]], [x[-1], y[-1]])
        constant = math.sqrt(a**2 + b**2)
        distance = np.abs(a*x + b*y + c) / constant
        drugThreshold[drug] = y[np.argmax(distance)]

In [9]:
prevalence = {}
totalCellline = {}
binaryDrugResponseDF = pd.DataFrame(np.nan, index = drugResponseDF.index, columns = drugResponseDF.columns)
for drug in binaryDrugResponseDF.columns:
    sens = drugResponseDF[drug] <= drugThreshold[drug]
    resi = drugResponseDF[drug] > drugThreshold[drug]
    if sum(sens) + sum(resi) > 200 and round(sum(sens) / (sum(sens) + sum(resi)), 4) >= 0.05:
        prevalence[drug] = round(sum(sens) / (sum(sens) + sum(resi)), 4)
        totalCellline[drug] = sum(sens) + sum(resi)
    binaryDrugResponseDF[drug][sens] = 1
    binaryDrugResponseDF[drug][resi] = 0
print("Number of valid drugs: ", len(totalCellline))
print("Average total number of celllines: ", np.mean(list(totalCellline.values())))
print("Average prevalence: ", np.mean(list(prevalence.values())))
print(binaryDrugResponseDF.shape)
binaryDrugResponseDF.to_csv(responseOutput, na_rep = "NA")
binaryDrugResponseDF.head()

Number of valid drugs:  352
Average total number of celllines:  821.7528409090909
Average prevalence:  0.24553267045454544
(953, 367)


,1,3,5,6,9,11,17,29,30,32,...,1494,1495,1496,1497,1498,1502,1526,1527,1529,1530
683665,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
683667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
684052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
684055,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
684057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


### 1.2 mutation data from a paper called "A Landscape of Pharmacogenomic Interactions in Cancer ". 

In [10]:
xl_file = pd.ExcelFile("GDSC_original_data/mmc3.xlsx")
dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}
print(dfs.keys())

/home/shr81/.conda/envs/MachineLearning/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


dict_keys(['TableS2A-CancerGenes', 'TableS2B-TumourVariants', 'TableS2C-CellLineVariants', 'TableS2D-RACSs', 'TableS2E-RACSsPanCanVSCanSpec', 'TableS2F-TumoursRACSs_CNA', 'TableS2G-CellLinesRACSs_CNA', 'TableS2H-InformativeCpGislands', 'TableS2I-TumoursHypMet_iCgPs', 'TableS2J-CellLinesHypMet_iCgPs', 'TableS2K-missingCFEs', 'TableS2L-CFEs_correlations', 'TableS2M-MergedGlobalPWcorrels'])


In [11]:
# This function could transfor excel table to matrix for mutation data and cna data
def excel_to_matrix_mutation(sheet_name,sample_start_point,sample_id_column_number,cancer_type_column_number,gene_column_number):
    variants_original_df = pd.DataFrame(data=dfs[sheet_name].iloc[sample_start_point:,[cancer_type_column_number,gene_column_number]].values,index=dfs[sheet_name].iloc[sample_start_point:,sample_id_column_number].values,columns=["cancer_type","gene"])
    unique_sample =variants_original_df.index.unique()
    unique_gene = variants_original_df.gene.unique()
    print("number of samples:",len(unique_sample),"number of genes:",len(unique_gene))
    variants_df = pd.DataFrame(data=np.zeros((len(unique_sample),len(unique_gene))),index=unique_sample,columns=unique_gene) #create an empty matrix with all zeros
    for sample_name in variants_df.index:
        res = variants_original_df.loc[sample_name,"gene"]
        mutated_genes = set([res]) if isinstance(res, str) else set(res.unique())    
        variants_df.loc[sample_name,mutated_genes] = 1 #get the mutation genes for each sample and set the these blocks to be 1
    variants_original_drop_duplicate_sample_df = variants_original_df[~variants_original_df.index.duplicated(keep='first')]
    variants_df["cancer_type"]= variants_original_drop_duplicate_sample_df.loc[variants_df.index,"cancer_type"] #add cancer type to the dataframe
    return variants_df,variants_df.columns

In [12]:
print("Cell line mutation information")
Cellline_mutation_cancer_type_df,cellline_mutation_gene_names = excel_to_matrix_mutation('TableS2C-CellLineVariants',20,2,3,4)
Cellline_mutation_cancer_type_df.shape

Cell line mutation information
number of samples: 1001 number of genes: 19100


(1001, 19101)

In [13]:
Cellline_mutation_df = Cellline_mutation_cancer_type_df.iloc[:,:-1]
Cellline_mutation_df

,A1BG,A1CF,A2M,A2ML1,A4D226,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
907272,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
998184,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
907289,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
905989,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
909698,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910209,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1240132,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1290807,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1330932,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 1.3 CNV data

In [14]:
GDSC_cnv_df = pd.read_csv("GDSC_original_data/WES_pureCN_CNV_genes_cn_category_20220623.csv",index_col=0) #cnv data

In [15]:
Cellline_info_raw_df = pd.read_csv("GDSC_original_data/model_list_20220810.csv",index_col=0)

In [16]:
# choose data source equals to sanger, remove .1 from sample name, and convert text to binarized form
cell_model_passport_cnv_T_df = GDSC_cnv_df.T
cell_model_passport_cnv_T_df = cell_model_passport_cnv_T_df[cell_model_passport_cnv_T_df["source"]=="Sanger"]
cell_model_passport_cnv_T_df.index = cell_model_passport_cnv_T_df.index.str.replace("\.1","")
cell_model_passport_cnv_T_df = cell_model_passport_cnv_T_df.iloc[:,3:] #remove column model_id, source, symbol
cell_model_passport_cnv_T_df = cell_model_passport_cnv_T_df.replace(["Neutral","Loss","Gain","Amplification","Deletion"],[0,0,0,1,1]) # "Neutral","Loss","Gain" to 0,amplification and deletion to 1

/home/shr81/.conda/envs/MachineLearning/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


In [17]:
#since mutation and expression data use cosmic_id, convert the cnv data from model name to cosmic_id
cell_line_info_df = Cellline_info_raw_df[["model_name",'COSMIC_ID']]
cell_line_info_df = cell_line_info_df.dropna()
cell_line_info_df = cell_line_info_df.set_index("model_name")
common_model_name_list = list(set(cell_model_passport_cnv_T_df.index)&set(cell_line_info_df.index))
print(len(common_model_name_list))
cell_line_info_df = cell_line_info_df.loc[common_model_name_list,:]
cell_model_passport_cnv_T_df =cell_model_passport_cnv_T_df.loc[common_model_name_list,:]

1008


### 1.4 SGA data processing

In [18]:
cell_model_passport_cnv_T_df = cell_model_passport_cnv_T_df.set_index(cell_line_info_df["COSMIC_ID"])
cell_model_passport_cnv_T_df = cell_model_passport_cnv_T_df.rename(index={'907284;1330932':'1330932'})
cell_model_passport_cnv_T_df = cell_model_passport_cnv_T_df.rename(index={'1331031;1331030':'1331030'})
cell_model_passport_cnv_T_df.index = cell_model_passport_cnv_T_df.index.astype(int)

common_sample_cnv_mut_list = list(set(Cellline_mutation_df.index)&set(cell_model_passport_cnv_T_df.index))
common_gene_cnv_mut_list = list(set(Cellline_mutation_df.columns)&set(cell_model_passport_cnv_T_df.columns))
# using common sample and gene to reconstruct these two dataframes
Cellline_mutation_df = Cellline_mutation_df.loc[common_sample_cnv_mut_list,common_gene_cnv_mut_list]
cell_model_passport_cnv_T_df = cell_model_passport_cnv_T_df.loc[common_sample_cnv_mut_list,common_gene_cnv_mut_list]
#if a gene is 1 in mutation or cnv, then it is 1, neither then 0
GDSC_mut_cnv_df = Cellline_mutation_df + cell_model_passport_cnv_T_df
GDSC_mut_cnv_arr = np.where(GDSC_mut_cnv_df.values>=1,1,0)
GDSC_mut_cnv_df = pd.DataFrame(index=GDSC_mut_cnv_df.index,columns=GDSC_mut_cnv_df.columns,data=GDSC_mut_cnv_arr)
GDSC_mut_cnv_df

,SDHB,PPP1R14D,WNK1,PEX26,ATXN7L3,VGF,PIP5K1C,GOLGA7,ANKRD50,ZNF35,...,MEP1B,CDKL1,HDGFL1,INSM2,TYMS,TRAM2,MIPEP,NAT1,CHRNA6,SLC25A15
907268,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
907269,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
907270,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
907271,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
907272,0,0,0,0,1,1,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753624,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
908135,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
688087,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
917486,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
GDSC_mut_cnv_df.index = [str(sample) for sample in GDSC_mut_cnv_df.index]

### 1.5 gene expression data from GDSC website

In [20]:
RNAseq_GDSC_df = pd.read_csv("GDSC_original_data/EXPmerged.csv",index_col=0)

In [95]:
new_column_name = [sample.replace("cosmic.","") for sample in RNAseq_GDSC_df.columns]
RNAseq_GDSC_df.columns = new_column_name
RNAseq_GDSC_T_df = RNAseq_GDSC_df.T
RNAseq_GDSC_T_df.head()

,ENSG00000000003,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,ENSG00000001460,...,ENSG00000278619,ENSG00000278662,ENSG00000278704,ENSG00000278828,ENSG00000278845,ENSG00000280071,ENSG00000280433,ENSG00000280670,ENSG00000280789,ENSG00000280987
1287381,5.742825,5.651001,4.467739,3.014166,-8.287284,7.213081,7.875378,7.328253,5.935010,3.913528,...,3.081384,1.984357,1.901941,-1.475922,5.893934,-0.778521,1.254001,0.463524,1.014700,3.523087
1287706,4.592277,5.974883,3.592532,5.099868,-5.318672,-3.159279,7.108233,4.926461,5.697733,3.430728,...,2.722252,0.274789,0.639703,-1.940189,4.372581,3.086884,0.500282,1.371473,2.619787,3.122676
910697,4.146911,6.570517,3.388971,2.786945,-1.732076,3.140121,6.143325,6.087315,6.018819,3.156478,...,2.799382,0.605541,-7.386796,-0.473561,4.977686,1.595625,0.968771,3.773703,3.666100,2.498545
910851,5.289480,5.993623,4.007116,4.056391,-3.789042,-2.565345,6.414339,4.548508,6.100855,3.962374,...,3.210041,1.059391,4.010217,-0.361289,6.496323,1.202010,1.153146,2.890040,2.867560,2.953157
910925,5.592378,6.567878,3.791862,5.487578,-2.851964,3.487037,7.073940,7.707461,5.538984,4.184304,...,3.447093,1.449922,0.460901,0.771684,5.855280,2.495384,0.577767,1.193354,3.131294,2.125035


In [96]:
GDSC_common_sample = list(set(GDSC_mut_cnv_df.index)&set(RNAseq_GDSC_T_df.index))
RNAseq_GDSC_T_df = RNAseq_GDSC_T_df.loc[GDSC_common_sample ]
GDSC_mut_cnv_df = GDSC_mut_cnv_df.loc[GDSC_common_sample ]
print(GDSC_mut_cnv_df.shape)
RNAseq_GDSC_T_df.head()

(976, 15692)


,ENSG00000000003,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,ENSG00000001460,...,ENSG00000278619,ENSG00000278662,ENSG00000278704,ENSG00000278828,ENSG00000278845,ENSG00000280071,ENSG00000280433,ENSG00000280670,ENSG00000280789,ENSG00000280987
908132,-2.864894,5.085821,4.109730,4.658060,-4.069375,-4.387285,-0.690449,5.680146,6.059928,3.010456,...,4.459784,-0.297243,-1.126221,0.658029,5.208837,3.999111,2.632519,1.822507,2.894252,3.449887
924106,5.901626,5.900997,4.851388,5.570010,-5.287576,-5.109632,4.231945,6.141810,6.330790,3.601342,...,2.655478,-0.433687,-0.342844,0.211684,5.983458,1.518033,1.317567,1.143826,4.431685,1.186881
909729,3.756783,6.518574,2.671258,4.709470,-7.838378,-6.145803,7.815655,6.299586,5.890664,3.315878,...,1.994625,-0.761175,2.085720,0.101980,4.730947,-1.524343,0.801560,1.154081,1.880756,2.556203
907276,5.643208,6.569087,3.257356,4.484493,-5.152704,-4.656031,7.782905,7.938462,5.241121,3.460099,...,4.619435,0.625857,2.331522,1.196622,5.845032,0.971824,0.031280,1.010198,3.586525,1.486064
946363,5.243715,5.622904,3.815669,5.111991,-6.986853,-1.020738,3.459126,5.805482,6.247374,2.247746,...,1.900902,1.794033,1.144987,-0.866314,5.482496,-0.184195,-0.116829,2.725201,2.984674,2.860856


In [97]:
#change the gene name from ensemble name to gene symbol, different version has some variance to define the gene name
mg = mygene.MyGeneInfo()
geneList = RNAseq_GDSC_T_df.columns
geneSyms = mg.querymany(geneList , scopes='ensembl.gene', fields='symbol', species='human')

INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:querying 1001-2000...
INFO:biothings.client:done.
INFO:biothings.client:querying 2001-3000...
INFO:biothings.client:done.
INFO:biothings.client:querying 3001-4000...
INFO:biothings.client:done.
INFO:biothings.client:querying 4001-5000...
INFO:biothings.client:done.
INFO:biothings.client:querying 5001-6000...
INFO:biothings.client:done.
INFO:biothings.client:querying 6001-7000...
INFO:biothings.client:done.
INFO:biothings.client:querying 7001-8000...
INFO:biothings.client:done.
INFO:biothings.client:querying 8001-9000...
INFO:biothings.client:done.
INFO:biothings.client:querying 9001-10000...
INFO:biothings.client:done.
INFO:biothings.client:querying 10001-11000...
INFO:biothings.client:done.
INFO:biothings.client:querying 11001-12000...
INFO:biothings.client:done.
INFO:biothings.client:querying 12001-13000...
INFO:biothings.client:done.
INFO:biothings.client:querying 13001-14000...
INFO:biothings

In [98]:
ensembl_gene_name = []
gene_symbol_name = []
for item in geneSyms:
    if 'symbol' in item:
        ensembl_gene_name.append(item['query'])
        gene_symbol_name.append(item['symbol'])
RNAseq_GDSC_T_df = RNAseq_GDSC_T_df[ensembl_gene_name]
RNAseq_GDSC_T_df.columns = gene_symbol_name
RNAseq_GDSC_T_df.head()

,TSPAN6,DPM1,SCYL3,FIRRM,FGR,CFH,FUCA2,GCLC,NFYA,STPG1,...,C11orf98,MRM1,GOLGA6L10,H3C10,MRPL45,LOC102724023,LOC102724200,CCDC163,PAGR1,MATR3
908132,-2.864894,5.085821,4.109730,4.658060,-4.069375,-4.387285,-0.690449,5.680146,6.059928,3.010456,...,2.791495,4.459784,-0.297243,0.658029,5.208837,3.999111,2.632519,1.822507,2.894252,3.449887
924106,5.901626,5.900997,4.851388,5.570010,-5.287576,-5.109632,4.231945,6.141810,6.330790,3.601342,...,2.759417,2.655478,-0.433687,0.211684,5.983458,1.518033,1.317567,1.143826,4.431685,1.186881
909729,3.756783,6.518574,2.671258,4.709470,-7.838378,-6.145803,7.815655,6.299586,5.890664,3.315878,...,3.775111,1.994625,-0.761175,0.101980,4.730947,-1.524343,0.801560,1.154081,1.880756,2.556203
907276,5.643208,6.569087,3.257356,4.484493,-5.152704,-4.656031,7.782905,7.938462,5.241121,3.460099,...,3.753014,4.619435,0.625857,1.196622,5.845032,0.971824,0.031280,1.010198,3.586525,1.486064
946363,5.243715,5.622904,3.815669,5.111991,-6.986853,-1.020738,3.459126,5.805482,6.247374,2.247746,...,2.730621,1.900902,1.794033,-0.866314,5.482496,-0.184195,-0.116829,2.725201,2.984674,2.860856


### 1.6 cancer type data

In [25]:
GDSC_cancer_type_df = Cellline_mutation_cancer_type_df.iloc[:,-1]
GDSC_cancer_type_df.index = [str(sample) for sample in GDSC_cancer_type_df.index]
GDSC_cancer_type_df = GDSC_cancer_type_df.loc[RNAseq_GDSC_T_df.index]
GDSC_cancer_type_df

908132     NaN
924106    BRCA
909729     GBM
907276    STAD
946363      NB
          ... 
688021    SCLC
971774     NaN
909255     NaN
949170      NB
906763     ALL
Name: cancer_type, Length: 976, dtype: object

# 2. TCGA dataset

### 2.1 mutation data from TCGA website

In [26]:
TCGA_mutation_df = pd.read_csv("TCGA_original_data/mc3.v0.2.8.PUBLIC.nonsilentGene.xena",sep = "\t",header = 0, index_col = 0, keep_default_na = False) # Downloaded from the UCSC Xena TCGA data portal
TCGA_mutation_df = TCGA_mutation_df.T
print(TCGA_mutation_df.shape)
TCGA_mutation_df.head()

(9104, 40543)


sample,UBE2Q2,CHMP1B,PSMA2P1,SHQ1P1,CPHL1P,SSXP10,REM1,TCOF1,NSRP1,OPA6,...,TULP2,OR1E5,RP11-390F4.3,GNGT2,GNGT1,PTRF,DIAPH2-AS1,SELV,NFIX,SELP
TCGA-02-0003-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-02-0033-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-02-0047-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-02-0055-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TCGA-02-2470-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [27]:
TCGA_mutation_df.shape

(9104, 40543)

### 2.2 CNV data from TCGA website

In [28]:
TCGA_cnv_ori_df = pd.read_csv("TCGA_original_data/Gistic2_CopyNumber_Gistic2_all_thresholded.by_genes",sep='\t',index_col=0)

### 2.3 SGA data processing 

In [29]:
TCGA_cnv_df = TCGA_cnv_ori_df.T
TCGA_cnv_df = TCGA_cnv_df.replace([-2,-1,0,1,2],[1,0,0,0,1]) # for cnv data, only set -2 and 2 to be 1, otherwise 0
TCGA_common_samples = list(set(TCGA_mutation_df.index)&set(TCGA_cnv_df.index))

In [30]:
TCGA_common_gene_mut_cnv_list = list(set(TCGA_mutation_df.columns)&set(TCGA_cnv_df.columns))
TCGA_cnv_df = TCGA_cnv_df.loc[TCGA_common_samples,TCGA_common_gene_mut_cnv_list]
TCGA_mutation_df = TCGA_mutation_df.loc[TCGA_common_samples,TCGA_common_gene_mut_cnv_list]

TCGA_mutation_df = TCGA_mutation_df.astype(float)
TCGA_mut_cnv_df = TCGA_cnv_df + TCGA_mutation_df
TCGA_mut_cnv_df = TCGA_mut_cnv_df.replace([0,1,2],[0,1,1])
TCGA_mut_cnv_df.head()

Sample,SDHB,PPP1R14D,ZNF66,ATXN7L3,MIR4263,SYTL1,RELL1,HECTD4,IL36G,LINC00621,...,UBE2Q2P2,LPHN3,RN7SL862P,RGS16,C4orf40,ADAT1,PEBP1,CDKL1,TRAM2,CHRNA6
TCGA-78-7542-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-G9-6367-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-S2-AA1A-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-2Z-A9JG-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCGA-OR-A5LO-01,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2.4 expression data from TCGA website

In [31]:
TCGA_xena_exprs_df = pd.read_csv("TCGA_original_data/EB++AdjustPANCAN_IlluminaHiSeq_RNASeqV2.geneExp.xena",sep="\t",index_col=0) # Downloaded from the UCSC Xena TCGA data portal
TCGA_xena_exprs_df.head()

,TCGA-OR-A5J1-01,TCGA-OR-A5J2-01,TCGA-OR-A5J3-01,TCGA-OR-A5J5-01,TCGA-OR-A5J6-01,TCGA-OR-A5J7-01,TCGA-OR-A5J8-01,TCGA-OR-A5J9-01,TCGA-OR-A5JA-01,TCGA-OR-A5JB-01,...,TCGA-CG-4449-01,TCGA-CG-4462-01,TCGA-CG-4465-01,TCGA-CG-4466-01,TCGA-CG-4469-01,TCGA-CG-4472-01,TCGA-CG-4474-01,TCGA-CG-4475-01,TCGA-CG-4476-01,TCGA-CG-4477-01
sample,,,,,,,,,,,,,,,,,,,,,
100130426,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100133144,2.09,1.88,1.45,0.00,0.00,1.12,1.29,0.00,2.45,3.02,...,2.42,2.74,2.64,3.99,4.40,3.00,4.27,3.71,4.29,4.24
100134869,2.30,3.32,2.92,1.35,2.45,2.82,1.72,1.24,2.60,0.00,...,1.87,2.12,1.78,2.60,3.66,3.87,3.07,3.65,3.64,3.99
10357,7.23,6.36,6.45,5.78,6.09,6.71,6.58,6.31,6.13,7.29,...,9.31,8.20,9.43,9.14,10.02,9.32,9.54,8.99,9.48,9.46
10431,10.99,10.35,10.04,11.20,10.30,10.87,9.96,10.78,10.25,10.97,...,10.23,9.33,10.21,9.38,9.34,10.34,10.17,10.43,9.93,10.76


In [32]:
TCGA_exprs_df = TCGA_xena_exprs_df.dropna().T
TCGA_exprs_df.shape

(11069, 16335)

In [33]:
TCGA_mut_cnv_df.shape

(8893, 23384)

In [34]:
TCGA_exprs_df.shape

(11069, 16335)

In [35]:
TCGA_common_sample_exprs_sga = list(set(TCGA_exprs_df.index)&set(TCGA_mut_cnv_df.index))
TCGA_exprs_processed_df = TCGA_exprs_df.loc[TCGA_common_sample_exprs_sga]
TCGA_mut_cnv_processed_df = TCGA_mut_cnv_df.loc[TCGA_common_sample_exprs_sga]
print(TCGA_exprs_processed_df.shape,TCGA_mut_cnv_processed_df.shape)

(8586, 16335) (8586, 23384)


### 2.5 cancer type data

In [36]:
TCGA_Study_Abbreviations_df = pd.read_excel("TCGA_original_data/TCGA_Study_Abbreviations.xlsx",index_col=1)
TCGA_Study_Abbreviations_new_index = [sample.lower() for sample in TCGA_Study_Abbreviations_df.index]
TCGA_Study_Abbreviations_new_index = [disease[:-1] if disease.endswith('s') else disease for disease in TCGA_Study_Abbreviations_new_index]
for i, element in enumerate(TCGA_Study_Abbreviations_new_index):
    if element == "uterine corpus endometrial carcinoma":
        TCGA_Study_Abbreviations_new_index[i] = "uterine corpus endometrioid carcinoma"
    if element == "cervical squamous cell carcinoma and endocervical adenocarcinoma":
        TCGA_Study_Abbreviations_new_index[i] = 'cervical & endocervical cancer'
    if element == "pheochromocytoma and paraganglioma":
        TCGA_Study_Abbreviations_new_index[i] = 'pheochromocytoma & paraganglioma'
    if element == "head and neck squamous cell carcinoma":
        TCGA_Study_Abbreviations_new_index[i] = 'head & neck squamous cell carcinoma'
    if element == "kidney renal papillary cell carcinoma":
        TCGA_Study_Abbreviations_new_index[i] = 'kidney papillary cell carcinoma'
    if element == "kidney renal clear cell carcinoma":
        TCGA_Study_Abbreviations_new_index[i] = 'kidney clear cell carcinoma'
    if element == "lymphoid neoplasm diffuse large b-cell lymphoma":
        TCGA_Study_Abbreviations_new_index[i] = 'diffuse large B-cell lymphoma'
    if element == "adrenocortical carcinoma":
        TCGA_Study_Abbreviations_new_index[i] = 'adrenocortical cancer'
TCGA_Study_Abbreviations_df.index = TCGA_Study_Abbreviations_new_index 

TCGA_Study_Abbreviations_df.head()

,Study Abbreviation
acute myeloid leukemia,LAML
adrenocortical cancer,ACC
bladder urothelial carcinoma,BLCA
brain lower grade glioma,LGG
breast invasive carcinoma,BRCA


In [37]:
TCGA_xena_cancertype = pd.read_csv("TCGA_original_data/TCGA_phenotype_denseDataOnlyDownload.tsv",sep="\t",index_col=0)
cancer_type = TCGA_Study_Abbreviations_df.loc[TCGA_xena_cancertype.loc[:,"_primary_disease"],"Study Abbreviation"].to_list()
TCGA_xena_cancertype["cancer_type"] = cancer_type
TCGA_cancer_type = TCGA_xena_cancertype.loc[TCGA_common_sample_exprs_sga,"cancer_type"].to_frame()
TCGA_cancer_type

,cancer_type
sample,
TCGA-78-7542-01,LUAD
TCGA-G9-6367-01,PRAD
TCGA-S2-AA1A-01,LUAD
TCGA-2Z-A9JG-01,KIRP
TCGA-OR-A5LO-01,ACC
...,...
TCGA-XJ-A83H-01,PRAD
TCGA-E8-A2EA-01,THCA
TCGA-XJ-A9DQ-01,PRAD


# 3. Combine GDSC and TCGA dataset

### 3.1 expression data combine

In [39]:
RNAseq_common_gene_gdsc_tcga = list(set(RNAseq_GDSC_T_df.columns)&set(TCGA_exprs_processed_df.columns))
print(len(RNAseq_common_gene_gdsc_tcga))

RNAseq_GDSC_T_df = RNAseq_GDSC_T_df[RNAseq_common_gene_gdsc_tcga]
RNAseq_GDSC_final_df = RNAseq_GDSC_T_df.loc[:,~RNAseq_GDSC_T_df.columns.duplicated()].copy() #drop duplicate column
TCGA_exprs_final_df = TCGA_exprs_processed_df[RNAseq_common_gene_gdsc_tcga]
print(RNAseq_GDSC_final_df.shape,TCGA_exprs_final_df.shape)

12519
(976, 12519) (8586, 12519)


In [52]:
gene_exprs_list = pd.read_csv("exprs_gene_list_GDSC_yifan_mike.txt",header=None)
common_gene_exprs = list(set(RNAseq_GDSC_final_df.columns)&set(gene_exprs_list.iloc[:,0].tolist()))
len(common_gene_exprs)

            0
0        A1BG
1        A1CF
2         A2M
3       AADAC
4      ABCA12
...       ...
2753    ZNF91
2754    ZNF92
2755      ZP3
2756  ZSCAN16
2757  ZSCAN18

[2758 rows x 1 columns]


2317

# Since the package "mygene" change the version with time, the gene name has a little change, this will result to different number of overlap genes.

In [132]:
GDSC_RNAseq_df = RNAseq_GDSC_final_df[common_gene_exprs]
TCGA_RNAseq_df = TCGA_exprs_final_df[common_gene_exprs]
print(GDSC_RNAseq_df.shape,TCGA_RNAseq_df.shape)

(976, 2317) (8586, 2317)


### 3.2 SGA data combine

In [41]:
SGA_common_gene_gdsc_tcga = list(set(TCGA_mut_cnv_processed_df.columns)&set(GDSC_mut_cnv_df.columns))
GDSC_mut_cnv_df = GDSC_mut_cnv_df[SGA_common_gene_gdsc_tcga]
TCGA_mut_cnv_processed_df = TCGA_mut_cnv_processed_df[SGA_common_gene_gdsc_tcga]
print(GDSC_mut_cnv_df.shape,TCGA_mut_cnv_processed_df.shape)

(976, 15692) (8586, 15692)


In [42]:
gene_sga_list = pd.read_csv("TCI_fondation1_driver_dataset_combine_gene_list.txt",header=None)
common_gene_sga = list(set(GDSC_mut_cnv_df.columns)&set(gene_sga_list.iloc[:,0].tolist()))
print(len(common_gene_sga))

1084


In [131]:
GDSC_sga_df = GDSC_mut_cnv_df[common_gene_sga]
TCGA_sga_df = TCGA_mut_cnv_processed_df[common_gene_sga]
print(GDSC_sga_df.shape,TCGA_sga_df.shape)

(976, 1084) (8586, 1084)
